#Libraries and Data

In [1]:
#import libraries
import numpy as np
import pandas as pd
import itertools
from prophet import Prophet
import yfinance
import pickle
import xgboost as xgb

def fbprobetModel(stock_name,input_data):
    input_data["High"]
    # Load the saved model from the file
    loaded_model = {}
    with open('../TrainedModel/prophet/{}_prophet_model_1.pkl'.format(stock_name), 'rb') as f:
        loaded_model = pickle.load(f)
    
    # Load the saved model from the file
    loaded_model_prophet = {}
    with open('../TrainedModel/prophet/{}prophet_model_2.pkl'.format(stock_name), 'rb') as f:
        loaded_model_prophet = pickle.load(f)
    
    data = {
    'ds' : [input_data["ds"]]
     }

    # Create a new DataFrame object
    data_fr = pd.DataFrame(data)
    data_fr.head()
    
    forecast = loaded_model_prophet.predict(data_fr)
    forecast
    
    #get some variables
    df_xgb = forecast.loc[:, ["trend", "weekly",  "multiplicative_terms","yhat"]]
    # Create a dictionary with data
    data = {
        'High': [input_data["High"]],
        'Low': [input_data["Low"]],
        'y': [220.600006],
        'Volume': [input_data["Volume"]],
        'trend': df_xgb.head(1).trend.values,
        'weekly':df_xgb.head(1).weekly.values,
        'multiplicative_terms': df_xgb.head(1).multiplicative_terms.values,
    }

    # Create a new DataFrame object
    df = pd.DataFrame(data)
    Future = xgb.DMatrix(df, label = df.y)

    f_predictions = pd.Series(loaded_model.predict(Future), name = "XGBoost")
    print(f_predictions)


/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [29]:
#load the datast
stock_name = "TATAPOWER"
data = yfinance.download (tickers = "TATAPOWER.NS",start="2023-06-06",
                               interval = "1d", group_by = 'ticker', auto_adjust = True)

data["Data"] = data.index
datetime_obj = np.datetime_as_string(data.tail(1).Data.values[0], unit='D')

input_data = {"High" : data.tail(1).High.values[0],"Low" : data.tail(1).Low.values[0], 
              "Volume" : data.tail(1).Volume.values[0],
              'ds':datetime_obj}


fbprobetModel("TATAPOWER",input_data)


[*********************100%***********************]  1 of 1 completed
0    221.303223
Name: XGBoost, dtype: float32
